# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [7]:
# Write your code below.

%load_ext dotenv
%dotenv 


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [8]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [14]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet"))


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [15]:
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
dd_px

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year
npartitions=49,,,,,,,,,
A,"datetime64[ns, America/New_York]",float64,float64,float64,float64,int64,float64,float64,int32
ABNB,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
YUM,...,...,...,...,...,...,...,...,...
YUM,...,...,...,...,...,...,...,...,...


In [16]:
# Write your code below.
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

import numpy as np
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    hi_lo_range = lambda x: (x['High'] - x['Low'])
))


C:\Users\gibra\AppData\Local\Temp\ipykernel_21416\1499452593.py:5: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


In [18]:
dd_feat.compute()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,Close_lag_1,returns,hi_lo_range
ticker,,,,,,,,,,,,
A,2024-01-02 00:00:00-05:00,137.965874,140.361975,137.686329,138.524963,1441600,0.0,0.0,2024,NaN,NaN,2.675646
A,2024-01-03 00:00:00-05:00,137.776184,137.776184,130.857431,130.947281,2074500,0.0,0.0,2024,138.524963,-0.054703,6.918753
A,2024-01-04 00:00:00-05:00,130.338270,131.286726,129.978853,130.787537,2446600,0.0,0.0,2024,130.947281,-0.001220,1.307873
A,2024-01-05 00:00:00-05:00,129.789147,131.745974,128.411380,130.348236,1394000,0.0,0.0,2024,130.787537,-0.003359,3.334594
A,2024-01-08 00:00:00-05:00,129.928930,133.353375,129.599464,133.163681,1311400,0.0,0.0,2024,130.348236,0.021599,3.753912
...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,2024-01-25 00:00:00-05:00,128.551783,129.027053,127.274475,128.937943,1871700,0.0,0.0,2024,128.165634,0.006026,1.752578
YUM,2024-01-26 00:00:00-05:00,128.640891,129.403315,127.403198,127.819061,1157000,0.0,0.0,2024,128.937943,-0.008678,2.000117
YUM,2024-01-29 00:00:00-05:00,127.828973,129.264694,127.026935,129.264694,1482100,0.0,0.0,2024,127.819061,0.011310,2.237759


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [26]:
# Write your code below.
feat_dt = dd_feat.compute()
feat_dt


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,Close_lag_1,returns,hi_lo_range
ticker,,,,,,,,,,,,
A,2024-01-02 00:00:00-05:00,137.965874,140.361975,137.686329,138.524963,1441600,0.0,0.0,2024,NaN,NaN,2.675646
A,2024-01-03 00:00:00-05:00,137.776184,137.776184,130.857431,130.947281,2074500,0.0,0.0,2024,138.524963,-0.054703,6.918753
A,2024-01-04 00:00:00-05:00,130.338270,131.286726,129.978853,130.787537,2446600,0.0,0.0,2024,130.947281,-0.001220,1.307873
A,2024-01-05 00:00:00-05:00,129.789147,131.745974,128.411380,130.348236,1394000,0.0,0.0,2024,130.787537,-0.003359,3.334594
A,2024-01-08 00:00:00-05:00,129.928930,133.353375,129.599464,133.163681,1311400,0.0,0.0,2024,130.348236,0.021599,3.753912
...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,2024-01-25 00:00:00-05:00,128.551783,129.027053,127.274475,128.937943,1871700,0.0,0.0,2024,128.165634,0.006026,1.752578
YUM,2024-01-26 00:00:00-05:00,128.640891,129.403315,127.403198,127.819061,1157000,0.0,0.0,2024,128.937943,-0.008678,2.000117
YUM,2024-01-29 00:00:00-05:00,127.828973,129.264694,127.026935,129.264694,1482100,0.0,0.0,2024,127.819061,0.011310,2.237759


In [31]:

feat_dt['Close_lag_10'] = feat_dt['Close'].rolling(10).mean()
feat_dt.head(15)


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,Close_lag_1,returns,hi_lo_range,Close_lag_10
ticker,,,,,,,,,,,,,
A,2024-01-02 00:00:00-05:00,137.965874,140.361975,137.686329,138.524963,1441600,0.0,0.0,2024,NaN,NaN,2.675646,NaN
A,2024-01-03 00:00:00-05:00,137.776184,137.776184,130.857431,130.947281,2074500,0.0,0.0,2024,138.524963,-0.054703,6.918753,NaN
A,2024-01-04 00:00:00-05:00,130.338270,131.286726,129.978853,130.787537,2446600,0.0,0.0,2024,130.947281,-0.001220,1.307873,NaN
A,2024-01-05 00:00:00-05:00,129.789147,131.745974,128.411380,130.348236,1394000,0.0,0.0,2024,130.787537,-0.003359,3.334594,NaN
A,2024-01-08 00:00:00-05:00,129.928930,133.353375,129.599464,133.163681,1311400,0.0,0.0,2024,130.348236,0.021599,3.753912,NaN
A,2024-01-09 00:00:00-05:00,132.055481,135.429989,129.799137,130.468048,1435000,0.0,0.0,2024,133.163681,-0.020243,5.630852,NaN
A,2024-01-10 00:00:00-05:00,130.368213,130.947274,128.151812,130.877380,1326300,0.0,0.0,2024,130.468048,0.003137,2.795462,NaN
A,2024-01-11 00:00:00-05:00,130.368215,130.468044,127.692561,129.469666,2060500,0.0,0.0,2024,130.877380,-0.010756,2.775482,NaN
A,2024-01-12 00:00:00-05:00,130.098655,131.396550,129.429743,130.328278,1285200,0.0,0.0,2024,129.469666,0.006632,1.966806,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
    Answer = No 

+ Would it have been better to do it in Dask? Why?
    Answer = Yes, Dask allows you to read larger datasets and multiple files in parallel

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.